# MyMethods.jl -- Examples

This notebook illustrates basic functionalities of the MyMethods.jl package.

### Installation and Uninstallation of MyMethods.jl

In [1]:
# Install the package from GitHub (first time only) 
using Pkg; 
Pkg.add(url="https://github.com/thomaswiemann/MyMethods.jl");

# Uninstall 
#Pkg.rm("MyMethods")

   Updating git-repo `https://github.com/thomaswiemann/MyMethods.jl`


Fetching: [========================================>]  100.0 %

   Updating registry at `C:\Users\thomas\.julia\registries\General`


LoadError: package `MyMethods [18de7e97]` has same name or UUID as the active project

In [2]:
# Load MyMethods.jl into your Julia session 
using MyMethods

┌ Info: Precompiling MyMethods [18de7e97-ce2a-4ea9-8d3b-e4b0c3b0b515]
└ @ Base loading.jl:1278
┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
│ - If you have MyMethods checked out for development and have
│   added DataFrames as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MyMethods
└ Loading DataFrames into MyMethods from project dependency, future warnings for MyMethods are suppressed.


In [3]:
# Load additionally useful (but not necessary) packages
using Statistics # for a quantile function
using Plots; plotly() # for a plotting backend

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\thomas\.julia\packages\Plots\D7Ica\src\backends.jl:373


Plots.PlotlyBackend()

### Exemplary Data  

Generate basic data to be used in the examples.

In [10]:
# Data parameters
N = 10000 # sample size
beta = [1,2,-1,4] # regression coefficients

# Data
_r = (1 .+ 1.5 .* (rand(N) .- 0.5)) # running variable
X = hcat(ones(N), rand(N,3)) # regressor matrix
y = X*beta.*_r + randn(N) # outcome simulation

10000-element Array{Float64,1}:
 11.78367408139462
  0.3683626477989679
  4.172372718740834
  2.349739701381286
  0.7467047097422491
  4.664548017250035
  5.504031687982217
  5.107816357778589
  2.013015467815926
  2.0070279299738254
  2.583894848584853
  1.4977323026832603
  4.8225191452993945
  ⋮
  1.877760216090623
  2.7309920626274717
  3.4288103602332844
  3.200904654213252
  2.2433482984283675
  7.635140974354565
  5.045731631878553
  4.245184476341436
  0.2556857014151921
  3.4488523774815114
  3.844638905510459
 -0.029567061265757477

### myLS
myLS() generates a (weighted) least squares object.

In [6]:
# Estimate least squares fit
fit_LS = myLS(y, X);

In [7]:
# Calculate and print standard errors
inference(fit_LS);

,coef,se,t-stat,p-val
,Float64,Float64,Float64,Float64
1,0.955292,0.0602235,15.8625,1.15295e-56
2,1.96151,0.0660663,29.69,1.033e-193
3,-0.941117,0.0659012,-14.2807,2.88529e-46
4,4.04962,0.0658136,61.5317,0.0


### myLLR
myLLR generates a local linear regression object.

In [20]:
# Estimate local constant regression with bandwidth h = 0.4
fit_LLR = myLLR(y, _r, # response and running variable
    X[:, 2:end],# additional variables to include
    _x = quantile(_r, collect(1:100)./100), # values at which to fit
    K=0, h=0.4); # degree and bandwidth

### continue from here
## revert pack to loadpath for easier debugging

In [20]:
# Calculate the local coefficients for quantiles of the running variable
coef_LLR = coef(fit_LLR);

In [21]:
# Plot coefficients as a function of the running variable
plot(fit_LLR._x, coef_LLR,
title = "Local linear regression coefficients",
label = ["Const." "X1" "X2" "X3"])

<!DOCTYPE html>
 
 
 Plots.jl

# mybootstrap

In [25]:
# bootstrap myLS
res = mybootstrap(myLS, y, 100, X, data_args = [1], 
    get = coef, red = hcat∘vcat);

In [24]:
# bootstrap myLLR
res = mybootstrap(myLLR, y, 10, _r, data_args = [1],
    get = coef) # think about bootstrapping myLLR!

LoadError: MethodError: no method matching Array{Float64,1}(::Array{Float64,2})
Closest candidates are:
  Array{Float64,1}(::AbstractArray{S,N}) where {T, N, S} at array.jl:562
  Array{Float64,1}() where T at boot.jl:425
  Array{Float64,1}(!Matched::UndefInitializer, !Matched::Int64) where T at boot.jl:406
  ...